# Szyfrowanie homomorficzne

### Zadanie 1
Dany jest szyfrogram otrzymany w wyniku działania algorytmu RSA (Textbook RSA encryption) na tekście jawnym o wartości 4. Na podstawie tego szyfroghramu wygeneruj fałszywy szyfrogram, który po odszyfrowaniu będzie równy 44. 

In [79]:
import random, sys, os

## Funkcje pomocnicze 
def gcd(a, b):
    # GCD - Greatest Common Divisor, Największy wspólny dzielnik 
    while a != 0:
        a, b = b % a, a   
    return b

def findModInverse(a, m):
    # Zwraca liczbę x odwrotną do a ciele skończonym modulo m 
    # czyli (a*x) % m =1 

    if gcd(a, m) != 1:
        return None #a i m muszą być względnie pierwsze aby istniał element odwrotny 

    # Rozszerzony algorytm Euklidesa 
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:   
        q = u3 // v3 # // operator dzielenie całkowitoliczbowego 
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

def generateKey(keySize):
    print('1. Generujemy liczby p i q')
    p=49109 
    q=40639
    e = 65537 
    print('3. Obliczamy wykładnik prywatny: odwrotność e modulo (p-1)(q-1)')
    d=findModInverse(e, (p-1)*(q-1))
    n= p*q
    publicKey = (n, e)
    privateKey = (n, d)
    print('Klucz publiczny:', publicKey)
    print('Klucz prywatny:', privateKey)
    return (publicKey, privateKey)

public, private = generateKey(16)
  
print('Generujemy klucze publiczny i prywatny:', public, private)

def encrypt(data_number, modulus, exp):
    data_encrypted = pow(data_number, exp, modulus)
    return data_encrypted

def decrypt(data_number, modulus, exp):
    data_decrypted = pow(data_number, exp, modulus)
    return data_decrypted

encrypted = encrypt(4, public[0], public[1])
print('Zaszyfrowana liczba:', encrypted)
decrypted = decrypt(encrypted, private[0], private[1])
print('Odszyfrowana liczba:', decrypted)

1. Generujemy liczby p i q
3. Obliczamy wykładnik prywatny: odwrotność e modulo (p-1)(q-1)
Klucz publiczny: (1995740651, 65537)
Klucz prywatny: (1995740651, 601158737)
Generujemy klucze publiczny i prywatny: (1995740651, 65537) (1995740651, 601158737)
Zaszyfrowana liczba: 350346157
Odszyfrowana liczba: 4


In [80]:
plaintext = 4
fake_text = 44

encrypted2 = encrypt(fake_text//plaintext, public[0], public[1])
print('Zaszyfrowana liczba:', encrypted)

decrypted2 = decrypt(encrypted2*encrypted, private[0], private[1])
print('Odszyfrowana liczba:', decrypted2)

Zaszyfrowana liczba: 350346157
Odszyfrowana liczba: 44


## Kryptosystem Pailliera



In [81]:
# !pip install phe

### Kryptosystem Pailliera

Q 1999 roku Pascal Paillier zaproponował kryptosystem zbliżony konstrukcją do RSA (bezpieczeństwo oparte o faktoryzację liczb całkowitych), które posiada własność szyfrowania homomorficznego dla operacji:
- dodawania
- mnożenia szyfrogramu przez jawny numer


#### Generowanie kluczy

1. Wybierz losowo dwie duże liczby pierwsze $p$ i $q$ tak aby $gcd(pq, (p-1)(q-1)) = 1$. 
2. Oblicz $n = pq$
3. Oblicz $\lambda = lcm(p-1,q-1)$ ($lcm$ --- Least Common Multiple, Najmniejsza Wspólna Wielokrotność)
4. Zdefiniuj funkcje $L(x) = \frac{x-1}{n}$
5. Wybierz losowo $g \in Z^*_{n^2}$ (liczba całkowita w zakresie 1 do $n^2$)
6. Oblicz odwrotność multiplikatywną $\mu = L(g^\lambda \bmod n^2))^{-1} bmod n$. Jeśli $\mu$ nie istnieje zacznij od nowa. 

Klucz publiczny ma postać: $pk = (n,g)$
Klucz prywatny ma postać: $sk = \lambda$ 

#### Szyfrowanie 

1. Tekstem jawnym jest liczba $m$ zakresu $0 \le m < n$.

2. Wybierz losową liczbę z zakresu $0 \le r < n$.

3. Oblicz szyfrogram $c= g^m \cdot r^n \bmod n^2$

#### Deszyfrowanie

1. Szyfrogram musi być liczbą z zakresu $0 < c < n^2$ 
2. Oblicz tekst jawny $m= L(c^\lambda \bmod n^2)\cdot \mu \bmod n$

### Homomorficzne własności schematu Pailliera

1. Dodawanie dwóch liczb:
$$D_{priv}(E_{pub}(m_1) \cdot E_{pub}(m_2) \bmod n^2)= m_1 + m_2 \bmod n$$

2. Mnożenie szyfrogramu przez liczbę:
$$D_{priv}(E_{pub}(m_1)^{m_2} \bmod n^2)= m_1 \cdot m_2 \bmod n$$

In [82]:
import phe  # https://pypi.org/project/phe/

from phe import paillier
import json
pub_key, priv_key = paillier.generate_paillier_keypair()
print(pub_key, priv_key)

enc1 = pub_key.encrypt(5)
enc2 = pub_key.encrypt(10)

print(enc1, enc2)

dec1 = priv_key.decrypt(enc1)
dec2 = priv_key.decrypt(enc2)

print(dec1, dec2)

<PaillierPublicKey a9fcece405> <PaillierPrivateKey for <PaillierPublicKey a9fcece405>>
<phe.paillier.EncryptedNumber object at 0x0000026E4902BBE0> <phe.paillier.EncryptedNumber object at 0x0000026E4902B5B0>
5 10


### Zadanie 2
Sprawdź empirycznie działanie homomorficznego szyfrowania Pailliera. Dlaczego to działa? Wykaż poprawność dodawania i mnożenia przez liczbę.   

In [83]:
decrypt_sum = priv_key.decrypt(10 + enc2)
print(decrypt_sum)

decrypt_prod = priv_key.decrypt(5 * enc2)
print(decrypt_prod)

20
50


Homomorficzne dodawanie dwóch wektorów

In [84]:
from Pyfhel import Pyfhel, PyPtxt, PyCtxt

import numpy as np

In [85]:
HE = Pyfhel()           # Creating empty Pyfhel object
HE.contextGen(scheme='bfv', n=2**14, t_bits=20)  # Generate context for 'bfv'/'ckks' scheme
                        # The n defines the number of plaintext slots.
                        #  There are many configurable parameters on this step
                        #  More info in Demo_2, Demo_3, and Pyfhel.contextGen()
HE.keyGen()             # Key Generation: generates a pair of public/secret keys
integer1 = np.array([127 for _ in range(2)], dtype=np.int64)
integer2 = np.array([-2, 12], dtype=np.int64)
ctxt1 = HE.encryptInt(integer1) # Encryption makes use of the public key
ctxt2 = HE.encryptInt(integer2) # For integers, encryptInt function is used.
print("3. Integer Encryption, ")
print("    int ",integer1,'-> ctxt1 ', type(ctxt1))
print("    int ",integer2,'-> ctxt2 ', type(ctxt2))


3. Integer Encryption, 
    int  [127 127] -> ctxt1  <class 'Pyfhel.PyCtxt.PyCtxt'>
    int  [-2 12] -> ctxt2  <class 'Pyfhel.PyCtxt.PyCtxt'>


In [86]:
ctxtSum = ctxt1 + ctxt2      
ctxtSub = ctxt1 * ctxt2


In [87]:
resSum = HE.decryptInt(ctxtSum) # Decryption must use the corresponding function
                                #  decryptInt.
resSub = HE.decryptInt(ctxtSub)
print("#. Decrypting result:")
print("     addition:       decrypt(ctxt1 + ctxt2) =  ", resSum.tolist()[:ctxt2.size()])
print("     substraction:   decrypt(ctxt1 - ctxt2) =  ", resSub.tolist()[:ctxt2.size()])


#. Decrypting result:
     addition:       decrypt(ctxt1 + ctxt2) =   [125, 139]
     substraction:   decrypt(ctxt1 - ctxt2) =   [-254, 1524]
